In [ ]:
import miditoolkit
import music21
import os
from miditoolkit import MidiFile


# =======================================================================

def convert_to_type0(midi_filename, new_filename=None):
    if new_filename is None:
        new_filename = midi_filename.replace('.mid', '_type0.mid')
    
    midi = MidiFile(midi_filename)
    all_instruments = sum([instrument.notes for instrument in midi.instruments], [])
    
    resulting_midi = midi
    track = miditoolkit.midi.containers.Instrument(program=0, is_drum=False, name='')
    track.notes = sorted(all_instruments, key=lambda x: (x.start, x.pitch))
    resulting_midi.instruments = [track]
    print(f"Converting {midi_filename} -> {new_filename}")
    
    return resulting_midi

In [ ]:
import pandas as pd

scores_df=pd.read_csv('../author-profiling-in-symbolic-music/dataframes/scores_df.csv')

paths_list=[]

for score_path in scores_df['scores_paths']:

    # Prepare the output path
    output_path = score_path.replace('../Lieder/scores/', '../author-profiling-in-symbolic-music/type 0 midis/')
    output_dir = os.path.dirname(output_path)
    os.makedirs(output_dir, exist_ok=True)  # Ensure the directory exists

    resulting_midi=convert_to_type0(score_path)
    # Dump each piano new score to a MIDI file
    output_file_name = f"{output_path}_type0.mid"
    resulting_midi.dump(output_file_name)
    paths_list.append(output_file_name)    


In [ ]:
type0_scores=scores_df
type0_scores['type0_path']=paths_list
type0_scores.to_csv('./dataframes/type0_scores.csv',index=False)

In [ ]:
##======= validating number of token sequences by score========##
from pathlib import Path
from symusic import Score
from miditok import REMI
tokenizer = REMI.from_pretrained("Natooz/Maestro-REMI-bpe20k")
input_midi_list=[]
gender_list=[]

for score_path,gender in zip(type0_scores['type0_path'],type0_scores['composer_gender']):

    input_midi_list.append(Score(Path(score_path)))
    gender_list.append(gender)


type0_scores=pd.DataFrame({'midi':input_midi_list,
              'gender':gender_list})

token_list=[]

for midi_score in type0_scores['midi']:

    input_tokens = tokenizer(midi_score)
    token_list.append(input_tokens)

type0_scores['tokens']=token_list